## Running the trained model on the test data

In [4]:
import json
import jsonlines
from nltk import sent_tokenize
from transformers import pipeline, AutoTokenizer,AutoModelForSeq2SeqLM

Initializing model and tokenizer:

In [5]:
model = AutoModelForSeq2SeqLM.from_pretrained('contest_train') #or any other path of yours
tokenizer = AutoTokenizer.from_pretrained('contest_train')

Opening the test data:

In [9]:
data = []

with jsonlines.open('test.jsonl', 'r') as f:
    for line in f:
        data.append(line['translation']['ru'])

In [10]:
data

['Эритроциты: отрицательно',
 'При осмотре обнаруживаются двусторонние припухлости вокруг моляров.',
 'Он испытывает замедленность движений и тремор.',
 'Она редко занимается спортом, и движение, похоже, дается ей с трудом.',
 'Нижние конечности нормальные, без отеков.',
 'У нее нет некоторых.',
 'Недавняя биопсия была отрицательной на гепатоцеллюлярную карциному и отличалась только хроническим воспалением, совместимым с хроническим вирусным гепатитом.',
 'Это женщина с ожирением, не имеющая острых расстройств.',
 'За последние 2 года у нее был только один партнер, и последние два года она не использовала никаких методов контрацепции.',
 'У него снова не было значительного ответа, хотя он продолжал иметь отличную работоспособность, и в конечном итоге его перевели на терапию Кэмпасом.',
 'Он живет со своей женой.',
 'Анализ крови показал повышенный уровень лактатдегидрогеназы и пониженный уровень человеческого комплемента C3 при нормальном профиле коагуляции.',
 'Он состоит в отношениях

Writing the preprocessing function:

In [7]:
def get_translation(text, lang, cur_model, cur_tokenizer):
    if lang == 'ru':
        translator = pipeline("translation_en_to_ru", model=cur_model, tokenizer=cur_tokenizer)
    else:
        translator = pipeline("translation_ru_to_en", model=cur_model, tokenizer=cur_tokenizer)
    text_length = len(text.split())
    text_split = str(text).split()
    if text_length > 100:
        sentences = sent_tokenize(text)
        result = []
        for sent in sentences:
            result.append(translator(sent)[0]['translation_text'])
        result = ' '.join(result)

    else:
        translation = translator(text)
        result = translation[0]['translation_text']    

    return result

Running the translation:

In [8]:
preds = []

for el in data:
    translated = get_translation(el, 'en', cur_model=model, cur_tokenizer=tokenizer)
    preds.append({"translation": {"ru": el, "en": translated}})

/home/ezherginovo/.local/lib/python3.8/site-packages/transformers/generation_utils.py:1839: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


KeyboardInterrupt: 

Saving the predictions into the desired format:

In [ ]:
with open('preds.json', 'w') as f:
      json.dumps(preds)